# Stratis Analysis: Sample Data from PIXNAT

2022 Oct 11

In [ ]:
import os,requests,shutil,time,getpass
from pyxnat import Interface
from glob import glob

In [ ]:
# URL of the XNAT host
xhost = 'http://pixnat.mskcc.org'

In [ ]:
# input XNAT credentials
xusr = 'locastre'
xpwd = getpass.getpass()

In [ ]:
# Authenticate to the XNAT via pyXNAT Interface
xnat = Interface(xhost,xusr,xpwd)

In [ ]:
session.auth = (xusr, xpwd)
auth = session.post(xhost)

In [ ]:
# Identify the project for the HNSCC test data
xproj = xnat.select.projects('HNSCC_Fuller*')
proj = xproj.get()
print(proj)

## Download the first 10 subjects from PIXNAT to data directory

In [ ]:
# Set local download data directory here
datadir = r'X:\Eve\radiomics-workflows'
print('User-set datadir ' + datadir + ' exists: ' + str(os.path.exists(datadir)))

In [ ]:
# Get RT_SIMULATION data from first 10 patients
subjlist = xproj.subjects('*').get()[0:10]
print(subjlist)

In [ ]:
analysis_list = []

for subj in subjlist:
    xsubj = xproj.subject(subj)
    xexplist = xsubj.experiments('*')
    for xexp in xexplist:
        session = xexp.attrs.get('label')
        exp_id = xexp.attrs.get('ID')        
        subjid = xexp.attrs.get('dcmPatientId')
        if 'RT_SIMULATION' in session:            
            expdir = os.path.join(datadir,subjid,session)
            analysis_list.append(expdir)
            print('Downloading session to folder: ' + expdir)
            os.makedirs(expdir,exist_ok=True)
            csv_file = os.path.join(expdir,'xnat.csv')
            f = open(csv_file,'w')
            f.write(subj + ',' + exp_id)
            f.close()
            xscanlist = xexp.scans('*')
            for xscan in xscanlist:
                scan = xscan.attrs.get('ID')
                modality = xscan.attrs.get('modality')
                scandir = os.path.join(expdir,modality,scan)
                os.makedirs(scandir,exist_ok=True)
                if modality == 'CT' or modality == 'RTDOSE':
                    res = 'DICOM'
                    resfilelist = xscan.resource(res).files('*').get()
                    for rfile in resfilelist:
                        xscan.resource(res).file(rfile).get(os.path.join(scandir,rfile))
                elif modality == 'RTPLAN' or modality == 'RTSTRUCT':
                    res = 'secondary'
                    resfilelist = xscan.resource(res).files('*').get()
                    for rfile in resfilelist:
                        xscan.resource(res).file(rfile).get(os.path.join(scandir,rfile))

## Perform CERR analysis here

In [ ]:
print('Folders for analysis: ')
print(analysis_list)

## Return RTSTRUCT results to PIXNAT

In [ ]:
import pydicom

In [ ]:
# rt_mod: helper function to correct possibly missing header info in RTSTRUCT

def rt_mod(dcmfile):
    d=pydicom.read_file(dcmfile)
    roi = os.path.join(os.path.dirname(dcmfile),'ROI_' + os.path.basename(dcmfile))
    print('ROI file out ' + roi)
    if d.StudyDate=='':
        d.StudyDate=d.StructureSetDate
    if d.StructureSetTime=='':
        d.StructureSetTime=time.strftime('%H%M%S') + '.00'
    if d.StudyTime=='':
        d.StudyTime=d.StructureSetTime
    try:
        agestr = d['PatientAge'].value
        if agestr != '':
            patientAge = int(agestr[:-1])
            if patientAge > 100:
                d['PatientAge'].value = '099Y'
    except:
        print('No PatientAge tag')
    d.save_as(roi)
    return(roi)

In [ ]:
headers = {'accept': 'text/plain','Content-Type': 'application/octet-stream'}

# Upload RTSTRUCT files as ROI Assessor to PIXNAT via direct REST call

for expdir in analysis_list:
    f = open(os.path.join(expdir,'xnat.csv'),'r')
    xnatinfo = f.readlines()[0].split(',')
    subj = xnatinfo[0]
    exp = xnatinfo[1]
    f.close()
    rtout = os.path.join(expdir,'output','RTSTRUCT.dcm') #needs updating to match analysis results
    rt_fixed = rt_mod(rtout)
    r = requests.post(req_URL)
    xlabel = subj + '_' + exp + '_STRATIS_HNSCC_demo'
    req_URL = xhost + '/xapi/roi/projects/' + proj + '/sessions/' + exp + '/collections/'+ xlabel + '?type=RTSTRUCT'
    with open(rt_fixed,'rb') as xstream:
        r = session.put(req_URL,data=xstream,headers=headers)

In [ ]:
# Close out user session on PIXNAT
xnat.disconnect()